In [1]:
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
import numpy as np

خواندن فایل و تبدیل به  متغیهای iran_geog_keywords، keyword و texts

In [4]:
# open file
keyword = open('sport_key_words.txt', 'r', encoding = 'utf-8').readlines()
texts = open('source.txt', 'r', encoding = 'utf-8').readlines()
geog_keywords = open('countries_cities.csv', 'r', encoding = 'utf-8').readlines()
iran_geog_keywords = open('city_iran.csv', 'r', encoding = 'utf-8').readlines()

max_allowded_space = 5

In [5]:
def read_file(file):
    texts = []
    for word in file:
        text = word.rstrip('\n')
        texts.append(text)
    return texts

key = read_file(keyword)
text = read_file(texts)
geog_keyword = read_file(geog_keywords+iran_geog_keywords) 

مجموعه ای از کلمات کلیدی در مورد 'ورزش' داریم.

اگر الگویی از این کلمات را بتوان در جمله ی ورودی یافت، در این صورت سراغ زیر گروه های می رویم (که روش آن را در تابع بیشتر توضیح خواهیم داد). 

In [318]:
sport_key_word = ''.join([i+'|' for i in key if len(i)>2])

types = ['برد و باخت و تساوی'  , 'صعود و سقوط و حذف' ,  'قهرمان و نایب قهرمان' , 'کسب مدال']

sub_key_1 = 'برد|باخت|تساوی' # type = 'برد و باخت و تساوی'
sub_key_2 = 'صعود|سقوط|حذف' # type = 'صعود و سقوط و حذف'
sub_key_3 = 'قهرمان|نایب قهرمان' # type = 'قهرمان و نایب قهرمان'
sub_key_4 = 'مدال' # type = 'کسب مدال'

key_verbs = r'است|کرد|شد|بود'

geog_key_word = ''.join([i.strip() +'|' for i in re.sub('\"|\'|\]|\[|u200c','',str(geog_keyword).lower().replace('\\',' ').replace('  ',' ')).split(',') if len(i)>2 ])

در این قسمت می خواهیم مکان را پیدا کنیم.

برای پیدا کردن مکان ما دو مشکل برخورد کردیم:

۱ - که برخی از مکان ها توسط parsi_io پیدا نمی شدند. برای این کار یک دیتاست کوچک از شهر های معروف شهر های کشور های جهان و ایران جمع کردیم. روش کار به این صورت است:

$~~~~~~$ ۱ - از parsi_io استفاده می کنیم، اگر مکانی پیدا نکرد به دیتاست رجوع می کنیم

$~~~~~~$ ۲ - به دیتاست جانبی مراجعه می کنیم و اگر مکان را یافت، از آن استفاده می کنیم.

$~~~~~~$ ۳ - در غیر این صورت، مقدار تهی را جایگذاری می کنیم.


۲ - در برخی از جملات بیش از یک کلمه به عنوان مکان معرفی می شد. در برخی از مواقع، تنها نام یک مکان برده شده اما مکان وقوع نیست برای این کار ما برای بهبود از وجود کلمه ی "در" داخل جمله استفاده کردیم که مراحل به صورت زیر است:

$~~~~~~$ ۱ - اگر تنها یک مکان داریم، کاری انجام نمی دهیم.

$~~~~~~$ ۲ - اگر بیش از یک کلمه به عنوان مکان انتخاب شده بود، کلمه ی "در" را در متن جستجو می کنیم و مکان شروع آن، جایگاه کلمه در جمله را بیان می کند.

برای مثال: جمله ی 'در جام جهانی والیبال در قزاقستان تیم ایران موفق به کسب مدال نقره شد.'
 مکان های پیدا شده: 
 
 ['قزاقستان', 'ایران'] 
 [24, 32, 37, 42]
 
 مکان "در" در جمله:
 [0, 2, 21, 23] 
 دو "در" در جمله وجود دارد.
         
$~~~~~~$ ۳ - اکنون برای هر "در" مشاهده شده در جمله، "جایگاه نقطه ی شروع مکان های پیدا شده، نسبت به کلمه ی 'در'" را پیدا می کنیم، به عبارت دیگر، مکان های پیدا شده توسط ما، چقدر با کلمه ی 'در' فاصله دارد؟ برای این کار از تفاضل نقطه ی شروع مکان ها و مکان 'در' استفاده می کنیم.(برای هر باری که 'در' داخل جمله وجود دارد، این کار را تکرار می کنیم) بدیهی است که اگر این مقدار منفی باشد، باید از آن صرفه نظر کرد('در تهران' مقدارش مثبت شده و این 'در' اشاره به مکان بودن 'تهران دارد' و 'تهران در' مقدارش منفی که یعنی این 'در' درکنار 'تهران' جهت نمایش مکان بودن نیامده، شاید 'در'های دیگر موجود در جمله، به مکان بودن 'تهران اشاره کند' ) در نهایت کلمه ای که کمترین فاصله تا 'در' را دارد و این فاصله کمتر از مقداری معین است(۱ + تعداد space های مجاز بین انتهای 'در' و شروع کلمه ی بعدی)، به عنوان مکان انتخاب می شود.  برای صرفه نظر کردن، مقدار طول جمله را آن نسبت می دهیم که در argmin هرگز انتخاب نشود. 

ادامه ی مثال:

برای 'در' اول:
[24 37]
این یعنی همه مکان ها بعد از این 'در' آمده اند و از طرفی فاصله ی آن با کلمات معرف مکان زیاد است. پس از این 'در' اطلاعات خاصی نتوانستیم بدست آوریم.منظورمان مکان 'ایران' است.

برای 'در' دوم:
[3 16]
این یعنی همه مکان ها بعد از این 'در' آمده اند و از طرفی فاصله ی آن با یکی از کلمات معرف مکان کم است. پس از آن کلمه ی مکانی، مکان وقوع است. که این مکان 'قزاقستان' است.

$~~~~~~$ ۴ - اگر فاصله ی تمام کلمات از 'در' بیشتر از حد مجاز بود یا به هر دلیلی از این روش نتوانستیم مکانی، یا مکان هایی را تشخیص بدهیم(جمله ی 'در شیراز، در تبریز' دو 'در' و هر کدام یک مکان دارند که قابل تشخیص است)، همان خروجی مرحله ی قبل را برمی‌گرداند. 

In [29]:
# find locations

from parsi_io.modules.address_extractions import AddressExtraction

def location_extraction(line):
    
    extractor = AddressExtraction()

    loc = {'address':[],'address_span':[]}

    # 'address' 'address_span'
    ex_result = extractor.run(line)

    if len(ex_result['address'])>0 :
        loc['address'] = ex_result['address']
        loc['address_span'] = ex_result['address_span']
    else:
        re_results = re.finditer(geog_key_word,line)
        for re_result in re_results:
            detected_loc = line[re_result.span()[0]:re_result.span()[1]]
            if len(detected_loc)>1:
                loc['address'].append(line[re_result.span()[0]:re_result.span()[1]])
                loc['address_span'] += [re_result.span()[0],re_result.span()[1]]
        
    # if more than 1 loc
    if len(loc['address'])>1:
        dar_in_loc = 'در'
        dar_span = []
        re_dar_results = re.finditer(dar_in_loc,line)
        for re_dar_result in re_dar_results:
            detected_dar = line[re_dar_result.span()[0]:re_dar_result.span()[1]]
            if len(detected_dar)>1:
                dar_span += [re_dar_result.span()[0],re_dar_result.span()[1]]
                
        real_locs = []
        real_locs_span = []
        for span in dar_span[::2]:
            temp = (np.array(loc['address_span'][::2])  - span)
            temp[temp<0] = len(line)
            
            if np.min(temp)!= len(line) and np.min(temp)< max_allowded_space :
    
                real_loc = np.argmin(temp)
                real_locs.append(loc['address'][real_loc])
                real_locs_span +=[loc['address_span'][2*real_loc],loc['address_span'][2*real_loc+1]]
        if len(real_locs)>0:        
            return dict({'address':real_locs,'address_span':real_locs_span})
    
    
    if len(loc['address']) == 0:
        loc['address'] = ['']
        loc['address_span'] = ['']
    return loc

برای پبدا کردن کلمات کلیدی، همان روش معمولی استفاده از RegEx را بکار گرفتیم. تا در وحله ی اول ببینیم آیا موضوع ورزشی هست یا خیر؟ سپس بازه ی وجود کلمات کلیدی را هم بیابیم.
برای پیدا کردن عبارت کلیدی، بازی 'شروع اولین کلمه ی کلیدی' تا 'انتهای آخرین کلمه ی کلیدی' را به عنوان مکان عبارت کلیدی در نظر گرفتیم.

نکته ای وجود دارد و آن هم این است که گاهی text به صورت "کسب مدال طلا" می آید و گاهی یک جمله به طور کامل "بازیکن هلندی تبار عنوان نایب قهرمان را کسب کرد" و افعال جزو عبارات کلیدی نیستند. برای این مسئله، RegEx ای از افعال پر کاربرد ساختیم و اگر فاصله ی 'شروع فعل'  تا 'انتهای عبارت کلیدی' از حدی کمتر بود، 'انتهای عبارت کلیدی' را به 'انتهای فعل' گسترش دادیم.

در نهایت بر اساس این که کلمات کدام زیر شاخه از شاخه ی اصلی بیشتر در عبارت پیدا شده، زیر شاخه ی آن را تعیین می کنیم.  

In [7]:
def type_text_extraction(line):
    
    line_key_start_span_list = []
    line_key_end_span_list = []
    line_keys = []

    keys = re.finditer(sport_key_word,line)
    
    is_it_about_sport = False
    for k in keys: 

        if (k.span()[1] - k.span()[0])>1: 
            is_it_about_sport = True
            
            line_keys.append(line[k.span()[0]:k.span()[1]])
            line_key_start_span_list += [k.span()[0]]
            line_key_end_span_list += [k.span()[1]]
    if not is_it_about_sport:
        return 0
    
    line_key_start_span = line_key_start_span_list[ np.argmin(line_key_start_span_list) ]
    line_key_end_span = line_key_end_span_list[ np.argmax(line_key_end_span_list) ]



    line_verbs_span = []
    verbs = re.finditer(key_verbs,line)

    for v in verbs:
        line_verbs_span = line[v.span()[0]:v.span()[1]]

        if (v.span()[1]-v.span()[0])>1:

            if v.span()[0] - line_key_end_span < 3*max_allowded_space: # 3 words
                line_key_end_span = v.span()[1]

    # detecting the type
    
    result_types = []

    result_types.append(len([i for i in re.finditer(sub_key_1,line) if (i.span()[1] - i.span()[0])>1]))
    result_types.append(len([i for i in re.finditer(sub_key_2,line) if (i.span()[1] - i.span()[0])>1]))
    result_types.append(len([i for i in re.finditer(sub_key_3,line) if (i.span()[1] - i.span()[0])>1]))
    result_types.append(len([i for i in re.finditer(sub_key_4,line) if (i.span()[1] - i.span()[0])>1]))

    return dict({'text':line[line_key_start_span : line_key_end_span] ,
                 'span':[line_key_start_span,line_key_end_span] ,
                 'type':types[np.argmax(result_types)]})

In [326]:
results = []
for line in text:
    
    result = dict({"line":line ,"type":'' ,"text":'' ,"span":None ,"place":'',"time":'' })
    
    # detecting locations
    
    loc = location_extraction(line)
    result['place'] = loc['address']
    
    # detecting key words
    
    type_text = type_text_extraction(line)
    result['type'] = type_text['type']
    result['span'] = type_text['span']
    result['text'] = type_text['text']
    results.append(result)
          


In [327]:
results

[{'line': 'در دیدار شب گذشته در مادرید تیم اتلتیکو مادرید به مرحله مسابقات صعود کرد.',
  'type': 'صعود و سقوط و حذف',
  'text': 'تیم اتلتیکو مادرید به مرحله مسابقات صعود کرد',
  'span': [28, 72],
  'place': ['مادرید'],
  'time': ''},
 {'line': 'بازیکن هلندی تبار عنوان نایب قهرمان را کسب کرد.',
  'type': 'قهرمان و نایب قهرمان',
  'text': 'بازیکن هلندی تبار عنوان نایب قهرمان را کسب کرد',
  'span': [0, 46],
  'place': [],
  'time': ''},
 {'line': 'در جام جهانی والیبال در قزاقستان تیم ایران موفق به کسب مدال نقره شد.',
  'type': 'کسب مدال',
  'text': 'جام جهانی والیبال در قزاقستان تیم ایران موفق به کسب مدال نقره شد',
  'span': [3, 67],
  'place': ['قزاقستان'],
  'time': ''}]